In [1]:
# Standard Libraries
import os
import math
import random

# Data Analysis and Visualization Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Interactive Data Visualization Libraries
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

# Data Analysis and Statistics Libraries
import scipy.stats as stats
from scipy.stats import skew, kurtosis, jarque_bera, shapiro, anderson

# Jupyter Notebook Display Libraries
from IPython.display import display, HTML

# Utility Libraries
from tqdm import tqdm as tqdm




##     ##               #######                                                                                                             
##      ##              #     #                                                                                                             
##       ##             #        #######  #######           #     #  #######  #######     #     #######  ######   #        #######  ####### 
##       ##             #######  #           #              #     #  #     #  #     #     #     #     #  #    #   #        #        #       
##       ##                  ##  ####        #               #   #   #######  #######     #     #######  #######  #        ####     ####### 
##      ##              #    ##  #           #                # #    #     #  #    #      #     #     #  #     #  #        #              # 
##     ##               #######  #######     #                 #     #     #  #    ##     #     #     #  #######  #######  #######  #######


# Define year range that should be incorporated
time_start = 2001
time_end = 2022

# Asset classes that are included either directly or as risk/return proxy into German Retail Investor Portfolio
retail_asset_classes = ['Listed Foreign Public Equity','Other Equity Instruments','State Bonds','Corporate Bonds','Listed Domestic Public Equity','Fixed Term and Savings Deposit','Public Investment Funds','Cash']
asset_classes_ex1 = ['Listed Foreign Public Equity','Other Equity Instruments','State Bonds','Corporate Bonds','Listed Domestic Public Equity','Fixed Term and Savings Deposit','Public Investment Funds','Cash']

# Define max weight per asset class for restricted portfolio optimization
max_weight = 0.33

# Define min cash for restricted portfolio optimization
cash_min = 0.12

# Define number of shots per Monte Carlo Simulation
n_unrestricted = 10000
n_restricted = 10000000

# Define risk free rate for Sharpe calc.
risk_free_rate=0.016 

# Activate model
active_classes = asset_classes_ex1.copy()



#######   ##               ######                                                                           #######                                                                                  
#######    ##              #                                                                                #     #                                                                                  
     ##     ##             #        #     #  #     #  #######  #######     #     #######  #     #           #     #  #######  #######     #     #     #     #     #######     #     #######  #     # 
#######     ##             #####    #     #  ##    #  #           #        #     #     #  ##    #           ##    #  #        #           #     ##    #     #        #        #     #     #  ##    # 
##          ##             ##       #     #  # ### #  #           #        #     #     #  # ### #           ##    #  ####     ####        #     # ### #     #        #        #     #     #  # ### # 
#######    ##              ##       #     #  #    ##  #           #        #     #     #  #    ##           ##    #  #        #           #     #    ##     #        #        #     #     #  #    ## 
#######   ##               ##       #######  #     #  #######     #        #     #######  #     #           #######  #######  #           #     #     #     #        #        #     #######  #     # 
                                                                                                                                                                                                      

# Calculate Bayes_Steins estimates using the shrikage factor w
def bayes_stein_returns(matrix, adjust=True):

    t, n = matrix.shape
    sample_means = matrix.mean(axis=0)
    ones = np.ones_like(sample_means)

    E = np.cov(matrix, rowvar=False)
    # Zellner & Chetty adjustment of sample cov matrix
    adj_Z_C = (float(t) - 1) / (t - n - 2)
    E *= round(adj_Z_C, 4) 
    I = np.linalg.inv(E)
    grand_mean = sample_means.dot(I).dot(ones.T) / ones.dot(I).dot(ones.T)
    diff = sample_means - grand_mean
    denominator = n + 2 + diff.dot(t*I).dot(diff.T)
    w = round((n + 2) / denominator, 4)
    print('The weighting coefficient in Bayes-Stein shrinkage: {}'.format(w))
    adjusted_means = (1 - w)*sample_means + w*grand_mean

    return adjusted_means

# Define a function to calculate CVaR at a given confidence level
def calculate_cvar(data, alpha):
    sorted_returns = np.sort(data)
    var_index = int((1 - alpha) * len(sorted_returns))
    cvar = -sorted_returns[:var_index].mean()
    return cvar

# Plot efficient frontier
def plot_ef  (mean_variance_pairs,tickers_list,weights_list,risk_free_rate,retail_return,retail_std,title_name): 
    #-- Plot the risk vs. return of randomly generated portfolios
    #-- Convert the list from before into an array for easy plotting
    mean_variance_pairs = np.array(mean_variance_pairs)


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                        marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                    showscale=True, 
                                    size=7,
                                    line=dict(width=1),
                                    colorscale="RdBu",
                                    colorbar=dict(title="Sharpe<br>Ratio")
                                    ), 
                        mode='markers',
                        text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))
    fig.update_layout(template='plotly_white',
                    xaxis=dict(title='Annualised Risk (Volatility)'),
                    yaxis=dict(title='Annualised Return'),
                    title=title_name,
                    width=1000,
                    height=700,
                    showlegend = False)

    #-- Find the portfolio with the highest Sharpe ratio
    sharpe_ratio = (mean_variance_pairs[:,0]-risk_free_rate) / (mean_variance_pairs[:,1]**0.5)
    max_sharpe_idx = np.argmax(sharpe_ratio)
    max_sharpe_weights = weights_list[max_sharpe_idx]
    max_sharpe_tickers = tickers_list[max_sharpe_idx]
    max_sharpe_return = np.round(mean_variance_pairs[max_sharpe_idx][0] * 100,decimals=2)
    max_sharpe_risk = np.round(mean_variance_pairs[max_sharpe_idx][1]**0.5 * 100,decimals=2)

    #-- Find the portfolio with the minimum variance
    min_var_idx = np.argmin(mean_variance_pairs[:,1])
    min_var_weights = weights_list[min_var_idx]
    min_var_tickers = tickers_list[min_var_idx]
    min_var_return = np.round(mean_variance_pairs[min_var_idx][0] * 100,decimals=2)
    min_var_risk = np.round(mean_variance_pairs[min_var_idx][1]**0.5 * 100,decimals=2)

    # Convert mean_variance_pairs_re into a NumPy array
    mean_variance_pairs_array = np.array(mean_variance_pairs)

    # Extract the standard deviation array
    std_array = np.sqrt(mean_variance_pairs_array[:, 1])

    # Find the index of the pair with the closest standard deviation
    closest_index = np.argmin(np.abs(std_array - retail_std))

    closest_std_tickers = tickers_list[closest_index]
    closest_std_weights = weights_list[closest_index]
    

    #-- Take the portfolio stats of the average German private houshold portfolio
    # Extract the index as an array
    retail_tickers = retail_asset_allocation.index.str.lower().to_numpy()
    retail_weights = retail_asset_allocation['Allocation'].to_numpy()

    #-- Add the portfolios to the scatter plot
    fig.add_trace(go.Scatter(x=[mean_variance_pairs[max_sharpe_idx,1]**0.5], y=[mean_variance_pairs[max_sharpe_idx,0]], 
                            marker=dict(color='green', size=10), 
                            mode='markers', 
                            name=None,
                            text=['Portfolio with highest Sharpe ratio (' + str(np.array(max_sharpe_tickers)) + ', ' + str(np.array(max_sharpe_weights).round(2)) + ')']))

    fig.add_trace(go.Scatter(x=[mean_variance_pairs[min_var_idx,1]**0.5], y=[mean_variance_pairs[min_var_idx,0]], 
                            marker=dict(color='red', size=10), 
                            mode='markers', 
                            name=None,
                            text=['Minimum variance portfolio (' + str(np.array(min_var_tickers)) + ', ' + str(np.array(min_var_weights).round(2)) + ')']))

    fig.add_trace(go.Scatter(x = [retail_std], y= [retail_return], 
                            marker=dict(color='purple', size=10), 
                            mode='markers', 
                            name=None,
                            text=['Retail Investor Portfolio (' + str(np.array(retail_tickers)) + ', ' + str(np.array(retail_weights).round(2)) + ')']))


    #-- Add the efficient frontier line from the risk-free rate to the portfolio with the highest Sharpe ratio
    ef_line_x = np.linspace(0, max(mean_variance_pairs[:,1]**0.5), 100)
    ef_line_y = ef_line_x * sharpe_ratio[max_sharpe_idx] + risk_free_rate
    fig.add_trace(go.Scatter(x=ef_line_x, y=ef_line_y, 
                            line=dict(color='black', width=2, dash='dash'),
                            name='Efficient Frontier'))



    # Plot the three relevant highlight portfolios with their assigned weights below
    max_sharpe_df = pd.DataFrame({'Asset_classes': max_sharpe_tickers, 'Weights': max_sharpe_weights})
    max_sharpe_df = max_sharpe_df.sort_values('Weights', ascending=False)
    max_sharpe_df['Weights'] = (max_sharpe_df['Weights'] * 100).round(2).astype(str) + '%'

    min_var_df = pd.DataFrame({'Asset_classes': min_var_tickers, 'Weights': min_var_weights})
    min_var_df = min_var_df.sort_values('Weights', ascending=False)
    min_var_df['Weights'] = (min_var_df['Weights'] * 100).round(2).astype(str) + '%'

    closest_std_df = pd.DataFrame({'Asset_classes': closest_std_tickers, 'Weights': closest_std_weights})
    closest_std_df = closest_std_df.sort_values('Weights', ascending=False)
    closest_std_df['Weights'] = (closest_std_df['Weights'] * 100).round(2).astype(str) + '%'
    closest_std_return = np.round(mean_variance_pairs[closest_index][0] * 100,decimals=2)
    closest_std_risk = np.round(mean_variance_pairs[closest_index][1]**0.5 * 100,decimals=2)

    # Transform retail portfolio ret and std. for plot
    retail_return = np.round(retail_return * 100,decimals=2)
    retail_std = np.round(retail_std * 100,decimals=2)

    retail_asset_allocation_df = retail_asset_allocation.copy()
    retail_asset_allocation_df = retail_asset_allocation_df.sort_values('Allocation', ascending=False)
    retail_asset_allocation_df['Allocation'] = (retail_asset_allocation['Allocation'] * 100).round(2).astype(str) + '%'
    retail_asset_allocation_df.reset_index(inplace=True)
    retail_asset_allocation_df.rename(columns={'Asset Class': 'Asset_classes', 'Allocation': 'Weights'},inplace = True)
    retail_asset_allocation_df['Asset_classes'] = retail_asset_allocation_df['Asset_classes'].str.lower()

    # remove index of max_sharpe_df and min_var_df
    max_sharpe_df = max_sharpe_df.reset_index(drop=True)
    min_var_df = min_var_df.reset_index(drop=True)
    closest_std_df = closest_std_df.reset_index(drop=True)


    # define CSS styles for the table
    styles = [
        dict(selector="th", props=[("text-align", "center"), ("font-family", "Poppins")]),
        dict(selector="td", props=[("text-align", "center"), ("font-family", "Poppins")]),
    ]

    # create HTML code for the dataframes with titles
    max_sharpe_html = f"<div style='display: inline-block; vertical-align: top; margin-right: 20px;'><h3 style='font-family: Poppins; font-size: 130%;'>Max Sharpe</h3>Return: {max_sharpe_return}%, Std.: {max_sharpe_risk}% <br><br>{max_sharpe_df.style.set_table_styles(styles).to_html(index=False)}</div>"
    min_var_html = f"<div style='display: inline-block; vertical-align: top; margin-right: 20px;'><h3 style='font-family: Poppins; font-size: 130%;'>Min Variance</h3>Return: {min_var_return}%, Std.: {min_var_risk}% <br><br>{min_var_df.style.set_table_styles(styles).to_html(index=False)}</div>"
    closest_std_html = f"<div style='display: inline-block; vertical-align: top; margin-right: 20px;'><h3 style='font-family: Poppins; font-size: 130%;'>Risk Matching Portfolio</h3>Return: {closest_std_return}%, Std.: {closest_std_risk}% <br><br>{closest_std_df.style.set_table_styles(styles).to_html(index=False)}</div>"
    retail_html = f"<div style='display: inline-block; vertical-align: top;'><h3 style='font-family: Poppins; font-size: 130%;'>Retail Portfolio</h3>Return: {retail_return}%, Std.: {retail_std}% <br><br>{retail_asset_allocation_df.style.set_table_styles(styles).to_html(index=False)}</div>"

    # Return the outputs
    return fig,max_sharpe_html,min_var_html,closest_std_html,retail_html

# function to load, transform and create data set
def data_loader (time_start,time_end,retail_asset_classes):


    #######   ##                                                                                                                 
    #######    ##                                                                                                                
        ##     ##                #     #     #  #######  #######  #######  #######           ######   #######  #######  ####### 
        ####     ##                #     ##   ##  #     #  #     #  #     #     #              #     #  #     #     #     #     # 
        ##     ##                #     # # # #  #######  #     #  #######     #              #     #  #######     #     ####### 
    #######    ##                 #     #  #  #  #        #     #  #    #      #              #     #  #     #     #     #     # 
    #######   ##                  #     #     #  #        #######  #    ##     #              ######   #     #     #     #     # 


    path = os.path.abspath("../Data")
    
    # Import quarterly index data from Preqin
    preqin_quarter_file = path + '/preqin_export_Private Capital_Quarterly_Index_Chart_2023_5_12.csv'
    preqin_data_quarter = pd.read_csv(preqin_quarter_file,delimiter=';', decimal=',')

    # Import Bloomberg data
    bloomberg_file = path + '/Bloomberg_selected_data.csv'
    bloomberg_data = pd.read_csv(bloomberg_file,delimiter=';', decimal=',')

    # Import DAX from Yahoo Finance
    dax_file = path + '/GDAXI.csv'
    dax_data = pd.read_csv(dax_file,delimiter=',', decimal='.')
    
    # Import SDAX
    sdax_file = path + '/SDAX_Historical_Data.csv'
    sdax = pd.read_csv(sdax_file,delimiter=',', decimal='.')

    # Import Fixed_term_and_savings_deposit returns from Deutsche Bundesbank
    savings_file = path + '/Fixed_Term_and_Savings_Deposit_Deutsche_Bundesbank.csv'
    savings_data = pd.read_csv(savings_file,delimiter=';', decimal=',')

    # Import Investment fund data from Rifinitiv
    Investment_fund_file = path + '/German_Investment_funds.csv'
    Investment_fund_data = pd.read_csv(Investment_fund_file,delimiter=';', decimal=',')


    # Create a dictionary with the asset allocation (incl. similar risk/return proxies) of German Private Housholds in 2022
    houshold_data = {
    'Asset Class': ['Listed Foreign Public Equity','Other Equity Instruments','State Bonds','Corporate Bonds','Listed Domestic Public Equity','Fixed Term and Savings Deposit','Public Investment Funds','Cash'],
    'Allocation': ['5%', '8.4%', '0.73%', '1.46%', '5.9%', '17.1%', '18.6%','42.8%']
    }

    # Create the dataframe
    retail_asset_allocation = pd.DataFrame(houshold_data)


    ##   ##   ##                                                                                                                                   
    ##   ##    ##                                                                                                                                  
    ##   ##     ##             ######   #######  #######  #######           #######  #        #######  #######  #     #     #     #     #  ####### 
    # #####     ##             #     #  #     #     #     #     #           #        #        #        #     #  ##    #     #     ##    #  #       
        ##     ##             #     #  #######     #     #######           #        #        ####     #######  # ### #     #     # ### #  #  #### 
        ##    ##              #     #  #     #     #     #     #           #        #        #        #     #  #    ##     #     #    ##  #     # 
        ##   ##               ######   #     #     #     #     #           #######  #######  #######  #     #  #     #     #     #     #  #######


    # PREQIN QUARTERLY DATA
    # drop the first row
    preqin_data_quarter = preqin_data_quarter.drop(0)

    # set the new column titles
    preqin_data_quarter.columns = preqin_data_quarter.iloc[0]
    preqin_data_quarter = preqin_data_quarter.drop(1)

    # reset the index
    preqin_data_quarter = preqin_data_quarter.reset_index(drop=True)

    preqin_data_quarter['DATE'] = pd.to_datetime(preqin_data_quarter['DATE'], format='%b-%y') + pd.offsets.MonthEnd(0)
    preqin_data_quarter['DATE'] = preqin_data_quarter['DATE'].dt.strftime('%d.%m.%Y')

    # set the date column as the index
    preqin_data_quarter = preqin_data_quarter.set_index('DATE')

    # convert the index to a DatetimeIndex
    preqin_data_quarter.index = pd.to_datetime(preqin_data_quarter.index)

    # replace commas with dots in all columns
    preqin_data_quarter = preqin_data_quarter.replace(',', '.', regex=True)

    # convert the values to numbers
    preqin_data_quarter = preqin_data_quarter.apply(pd.to_numeric)

    # calculate the quarterly returns
    preqin_quarterly_returns = preqin_data_quarter.resample('Q').last().pct_change()

    # drop the first row of the quarterly_returns DataFrame
    preqin_quarterly_returns = preqin_quarterly_returns.drop(preqin_quarterly_returns.index[0])

    # Filter rows based on year condition
    # Convert start and end to pd.datetime format
    preqin_start = pd.to_datetime(str(time_start))
    preqin_end = pd.to_datetime(str(time_end+1))

    # Create a boolean mask based on the condition
    mask = (preqin_quarterly_returns.index >= preqin_start) & (preqin_quarterly_returns.index <= preqin_end)

    # Filter the DataFrame using the boolean mask
    preqin_quarterly_returns = preqin_quarterly_returns[mask]


    # Convert the "Date" column to datetime
    bloomberg_data['Date'] = pd.to_datetime(bloomberg_data['Date'], format='%d.%m.%Y')

    # Set the "Date" column as the index
    bloomberg_data.set_index('Date', inplace=True)

    # Convert the index to a DatetimeIndex
    bloomberg_data.index = pd.to_datetime(bloomberg_data.index)

    # Filter rows based on year condition
    # Convert start and end to pd.datetime format
    x = pd.to_datetime(str(time_start))
    y = pd.to_datetime(str(time_end+1))

    # Create a boolean mask based on the condition
    mask = (bloomberg_data.index >= x) & (bloomberg_data.index <= y)

    bloomberg_data = bloomberg_data[mask]

    bloomberg_data.drop(bloomberg_data.columns[[0, 2]], axis=1, inplace=True)

    # Rename the columns
    bloomberg_data.rename(columns={
        '% Change': 'Bloomberg Euro Aggregate Corporate Total Return Index EU',
        '% Change.1': 'Bloomberg Euro Aggregate Treasury Germany Total Return Index'
    }, inplace=True)

    # Divide all columns by 100
    bloomberg_data = bloomberg_data / 100

    # Invert rows
    bloomberg_data = bloomberg_data.iloc[::-1]
    bloomberg_returns = bloomberg_data.copy()


    # DAX DATA
    # First, we need to convert the "Date" column to a datetime type
    dax_data['Date'] = pd.to_datetime(dax_data['Date'])

    # Set the "Date" column as the index of the DataFrame
    dax_data.set_index('Date', inplace=True)

    # Create an empty DataFrame to store the monthly returns
    dax_monthly_rets = pd.DataFrame(columns=['Year', 'DAX'])

    # Calculate and populate the monthly returns
    months = dax_data.resample('M').agg({'Close': ['last']})
    # Shift the 'Close' column one row to get the previous month's 'Close' price
    months['Previous_Close'] = months['Close'].shift(1)

    # Calculate the monthly return as the percentage change in 'Close' prices
    months['DAX'] = (months['Close','last'] / months['Previous_Close'] - 1)
    months.drop(columns=['Previous_Close','Close'], inplace=True)

    dax_monthly_rets = months.copy()

    # Filter rows based on year condition
    # Convert start and end to pd.datetime format
    x = pd.to_datetime(str(time_start))
    y = pd.to_datetime(str(time_end+1))

    # Create a boolean mask based on the condition
    mask = (dax_monthly_rets.index >= x) & (dax_monthly_rets.index <= y)

    # Filter the DataFrame using the boolean mask
    dax_monthly_rets = dax_monthly_rets[mask]

    # Remove the second heading level of the multi-level index
    dax_monthly_rets.columns = dax_monthly_rets.columns.droplevel(1)
    
    pd.set_option('display.max_rows', None)

    # SDAX DATA
    # Step 1: Select the first two columns
    sdax = sdax[['Date', 'Price']]

    # Step 2: Parse the Date column to datetime
    sdax['Date'] = pd.to_datetime(sdax['Date'], format='%m/%d/%Y')

    # Step 3: Set the Date column as the index
    sdax.set_index('Date', inplace=True)

    # Step 4: Remove commas and convert Price column to numeric
    sdax['SDAX'] = pd.to_numeric(sdax['Price'].str.replace(',', ''), errors='coerce')

    # Drop the original 'Price' column
    sdax.drop(columns=['Price'], inplace=True)

    # Step 5: Calculate monthly returns
    sdax_monthly_returns = sdax['SDAX'].resample('M').first().pct_change()

    # Adjust the monthly returns to represent the return for the current month
    sdax_monthly_returns.index = sdax_monthly_returns.index + pd.DateOffset(months=-1)

    # Step 6: Filter rows based on the year condition
    # Convert start and end to pd.datetime format
    x = pd.to_datetime(str(time_start), format='%Y')
    y = pd.to_datetime(str(time_end+1), format='%Y')

    # Create a boolean mask based on the condition
    temp_mask = (sdax_monthly_returns.index >= x) & (sdax_monthly_returns.index <= y)

    # Filter the DataFrame using the boolean mask
    sdax_monthly_rets = sdax_monthly_returns[temp_mask]




    # SAVINGS AND DEPOSIT DATA
    # Convert Year and Month columns to datetime format and add last day of the month
    savings_data['Date'] = pd.to_datetime(savings_data[['Year', 'Month']].assign(day=1)).dt.to_period('M').dt.to_timestamp('M') + pd.offsets.MonthEnd(0)

    # Calculate savings_returns based on the provided formula
    savings_data['savings_returns'] = (savings_data['Return, % p.a.'] / 12) * 0.01

    # Select only the Date and savings_returns columns for the final DataFrame
    savings_monthly = savings_data[['Date', 'savings_returns']]

    # Set the 'Year' column as the index
    savings_monthly = savings_monthly.set_index('Date')

    # Create a boolean mask based on the condition
    mask = (savings_monthly.index >= x) & (savings_monthly.index <= y)
    savings_monthly = savings_monthly[mask]


    # Investment Fund
    # Delete the first row
    Investment_fund_data = Investment_fund_data.iloc[1:]

    # Typecast the first column to pd.datetime and name it 'date'
    Investment_fund_data['Date'] = pd.to_datetime(Investment_fund_data.iloc[:, 0], format='%d-%b-%Y')

    # Set 'date' column as the index
    Investment_fund_data.set_index('Date', inplace=True)

    # Delete the first column where the old date format is stored‘
    Investment_fund_data = Investment_fund_data.iloc[:, 1:]

    # Create a new column named 'Investment_Funds' with average return per row
    Investment_fund_data['Investment_Funds'] = Investment_fund_data.mean(axis=1)

    investment_fund_return = pd.DataFrame(Investment_fund_data['Investment_Funds'])

    investment_fund_return = investment_fund_return.drop(investment_fund_return.index[0])
    investment_fund_return = investment_fund_return.drop(investment_fund_return.index[0])

    # Filter rows based on time condition
    investment_fund_return = investment_fund_return.loc[(investment_fund_return.index >= x)& (investment_fund_return.index <= y)]



    #######   ##                                                                                                                                                                       
    #######    ##                                                                                                                                                                      
    #           ##             ######   #######  #######  #######           #######  #######  #######           #######  #######  #######  #######  #######     #     #######  #     # 
    #######     ##             #     #  #     #     #     #     #           #        #           #              #        #     #  #        #     #     #        #     #     #  ##    # 
        ##     ##             #     #  #######     #     #######           #######  ####        #              #        #######  ####     #######     #        #     #     #  # ### # 
    #######    ##              #     #  #     #     #     #     #                 #  #           #              #        #    #   #        #     #     #        #     #     #  #    ## 
    #######   ##               ######   #     #     #     #     #           #######  #######     #              #######  #    ##  #######  #     #     #        #     #######  #     # 




    ## SET Creation ##

    ## Monthly return set##
    dax_monthly_rets = dax_monthly_rets.reset_index()

    # Reset index and drop "Date" column for sdax
    sdax_monthly_rets = sdax_monthly_rets.reset_index().drop(columns=['Date'])

    # Reset index and drop "Date" column for investment_fund_return
    investment_fund_return_reset = investment_fund_return.reset_index().drop(columns=['Date'])

    # Reset index and drop "Date" column for bloomberg_returns
    bloomberg_returns_reset = bloomberg_returns.reset_index().drop(columns=['Date'])

    # Reset index and drop "Date" column for savings_returns
    savings_monthly_reset = savings_monthly.reset_index().drop(columns=['Date'])

    # Concatenate the columns of the three dataframes
    monthly_returns = pd.concat([dax_monthly_rets, sdax_monthly_rets, investment_fund_return_reset, bloomberg_returns_reset,savings_monthly_reset], axis=1)



    ## Quarterly return set##

    # Step 2: Set the 'Date' column as the index
    monthly_returns.set_index('Date', inplace=True)

    combined_quarterly = monthly_returns.groupby(pd.Grouper(freq='Q')).apply(lambda x: (1 + x).prod() - 1)

    # Preqin-----------
    # Reset index and drop "Date" column for bloomberg_returns
    preqin_quarterly_returns_reset = preqin_quarterly_returns.reset_index().drop(columns=['DATE'])

    # Concatenate the columns of the three dataframes

    combined_quarterly_reset = combined_quarterly.reset_index()
    quarterly_returns = pd.concat([combined_quarterly_reset, preqin_quarterly_returns_reset], axis=1)

    # Add CASH as a 0-constant
    quarterly_returns['CASH'] = 0.00


    # Define a dictionary to map the old column names to the new column names
    column_mapping = {
    'S&P 500 TOTAL RETURN': 'Listed Foreign Public Equity',
    'SDAX': 'Other Equity Instruments',
    'Bloomberg Euro Aggregate Treasury Germany Total Return Index': 'State Bonds',
    'Bloomberg Euro Aggregate Corporate Total Return Index EU': 'Corporate Bonds',
    'DAX': 'Listed Domestic Public Equity',
    'savings_returns': 'Fixed Term and Savings Deposit',
    'Investment_Funds': 'Public Investment Funds',
    'CASH': 'Cash'
    }

    # Rename the columns using the column_mapping dictionary
    quarterly_returns = quarterly_returns.rename(columns=column_mapping)


    ## Annual return set ##


    # Create annual return set by grouping by year and summing the quarterly returns to get annual returns, taking into account the compounding effect within each year.
    # Set the 'Date' column as the index
    quarterly_returns.set_index('Date', inplace=True)

    annual_returns = quarterly_returns.groupby(pd.Grouper(freq='Y')).apply(lambda x: (1 + x).prod() - 1)

    # Convert index to datetime format and extract year
    annual_returns.index = pd.to_datetime(annual_returns.index).year

    # Rename the index to 'Year'
    annual_returns.index.name = 'Year'


    # GERMAN RETIAL INVESTOR PORTFOLIO CREATION


    # Select the desired columns from the dataframe
    retail_annual_returns = annual_returns[retail_asset_classes]

    # Transform the Allocation column to floats
    retail_asset_allocation['Allocation'] = retail_asset_allocation['Allocation'].str.rstrip('%').astype(float) / 100

    # Set 'Asset Class' as the index
    retail_asset_allocation.set_index('Asset Class', inplace=True)

    # Calculate weighted returns
    weighted_returns = retail_annual_returns.mul(retail_asset_allocation['Allocation'], axis=1)

    # Calculate the weighted average return per year
    retail_annual_returns['RETURN'] = weighted_returns.sum(axis=1)

    # Calculate standard deviation and mean return
    retail_std = retail_annual_returns['RETURN'].std()
    retail_return = retail_annual_returns['RETURN'].mean()

    return annual_returns, quarterly_returns, retail_asset_allocation, retail_annual_returns, retail_return, retail_std


#######   ##                                                                                                                                                                                                                    
#######    ##                                                                                                                                                                                                                   
##          ##             ######   #######  #######  #######  #######     #     #######  #######     #     #     #  #######           #######  #######  #######  #######     #     #######  #######     #     #######  ####### 
#######     ##             #     #  #        #        #        #     #     #     #     #     #        #     #     #  #                 #           #     #     #     #        #     #           #        #     #        #       
##   ##     ##             #     #  ####     #######  #        #######     #     #######     #        #      #   #   ####              #######     #     #######     #        #     #######     #        #     #        ####### 
#######    ##              #     #  #              #  #        #    #      #     #           #        #       # #    #                       #     #     #     #     #        #           #     #        #     #              # 
#######   ##               ######   #######  #######  #######  #    ##     #     #           #        #        #     #######           #######     #     #     #     #        #     #######     #        #     #######  ####### 

# Load data set
annual_returns, quarterly_returns, retail_asset_allocation, retail_annual_returns, retail_return, retail_std = data_loader(time_start,time_end,retail_asset_classes)

# Define colors for the chart
colors = ['#FFDC26', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#1F77B4', '#17becf', '#7F7F7F']

# Plot the retail asset allocation chart
fig = go.Figure(data=[go.Pie(labels=retail_asset_allocation.index, values=retail_asset_allocation['Allocation'], hole=0.4, marker=dict(colors=colors))])

# Set the aspect ratio to make the plot square-sized
fig.update_layout(
    width=600,
    height=600,
    legend=dict(orientation='h', y=-0.2)
)
# Set the axis labels and plot title
fig.update_layout(
    title='Retail Allocation Plot'
)

fig.show()

# Drop Return column
retail_annual_returns.drop(columns=['RETURN'],inplace=True)

# Convert the column names (x-axis labels) to lowercase for retail_annual_returns
annual_returns.columns = annual_returns.columns.str.lower()
quarterly_returns.columns = quarterly_returns.columns.str.lower()

# Select asset classes
active_classes = [element.lower() for element in active_classes]
selected_returns = annual_returns[active_classes]
selected_quarterly_returns = quarterly_returns[active_classes]

# Calculate summary statistics for each asset class
stats = pd.DataFrame(index=['mean', 'median', 'std', 'skewness', 'kurtosis'])
for col in selected_quarterly_returns.columns:
    selected_quarterly_returns[col] = selected_quarterly_returns[col].astype(float)
    selected_returns[col] = selected_returns[col].astype(float)
    mean = selected_returns[col].mean()
    median = selected_returns[col].median()
    std = selected_quarterly_returns[col].std() * np.sqrt(4)
    skewness_value = skew(selected_quarterly_returns[col])
    kurtosis_value = kurtosis(selected_quarterly_returns[col])
    stats[col] = [mean, median, std, skewness_value, kurtosis_value]

# Convert the column names (x-axis labels) to lowercase
stats.columns = stats.columns.str.lower()

# How many assests in the portfolio
n_assets = selected_returns.shape[1]

# transpose the table
stats_selected = stats.T
# Rename Index to asset_class
stats_selected.rename_axis('asset_class',inplace=True)
mus = (stats_selected['mean'])

# Covariance matrix of the asset classes based on annualized quarterly returns
cov = selected_quarterly_returns.cov() * 4

# Create a scatter plot for return/risk analysis
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=stats_selected['std'],
    y=stats_selected['mean'],
    mode='markers',
    marker=dict(
        size=10,
        color='blue',
        symbol='circle'
    )
))

# Set the aspect ratio to make the plot square-sized
fig.update_layout(
    width=600,
    height=600
)

# Set the axis labels and plot title
fig.update_layout(
    xaxis=dict(title='Standard Deviation'),
    yaxis=dict(title='Mean'),
    title='Risk-Return Plot'
)

# Add annotations for each asset class
annotations = [
    dict(
        x=x_val,
        y=y_val,
        xref="x",
        yref="y",
        text=asset,
        showarrow=True,
        arrowhead=7,
        ax=0,
        ay=-40
    )
    for asset, x_val, y_val in zip(stats_selected.index, stats_selected['std'], stats_selected['mean'])
]

fig.update_layout(
    annotations=annotations
)

# Show the plot
fig.show()


# Create the line plot
fig = go.Figure()

for column in selected_returns.columns[1:]:
    fig.add_trace(go.Scatter(x=selected_returns.index, y=selected_returns[column], mode='lines', name=column,))

# Customize the layout
fig.update_layout(
    title='Returns Over Time',
    xaxis_title='Year',
    yaxis_title='Return',
    legend_title='Asset Class',
    hovermode='x',
    height=600,
    xaxis=dict(
        type='date',
        tickformat='%Y',
        dtick='M12'
    )
)

# Display the plot
fig.show()


# Return estimators

selected_means = selected_returns.mean()
selected_medians = selected_returns.median()

# Calculate the annual geometric mean for each asset
geometric_means = (selected_returns + 1).prod()**(1 / len(selected_returns)) - 1

# Bayes-Stein (shrinkage)
# drop cash as 0 const otherwise the matrix cannot be inverted
# Find the index number of the column 'cash'
index_cash = selected_returns.columns.get_loc('cash')
selected_returns_wo_cash = selected_returns.drop(columns='cash')
selected_bayes_stein_returns = bayes_stein_returns(selected_returns_wo_cash)
# Create a new Series with the "cash" asset class and its corresponding return value
cash_series = pd.Series([0.0], index=["cash"])
# Concatenate the new_series with the original selected_bayes_stein_returns series
selected_bayes_stein_returns = pd.concat([selected_bayes_stein_returns.iloc[:index_cash], cash_series, selected_bayes_stein_returns.iloc[index_cash:]])


# Risk estimators

selected_skewness = selected_returns.skew()
selected_kurtosis = selected_returns.kurtosis()
selected_std_dev = selected_quarterly_returns.std() * np.sqrt(4)
selected_cvar = selected_returns.apply(calculate_cvar, alpha=0.9)


# Normality Tests
# Perform Jarque-Bera test
selected_jb = selected_returns.apply(lambda x: jarque_bera(x)[0])
selected_jb_p = selected_returns.apply(lambda x: jarque_bera(x)[1])
# Perform Shapiro-Wilk test
selected_sw = selected_returns.apply(lambda x: shapiro(x)[0])
selected_sw_p = selected_returns.apply(lambda x: shapiro(x)[1])
# Perform Anderson-Darling test
selected_ad = selected_returns.apply(lambda x: anderson(x, dist='norm')[0])
# Create an empty dictionary to store the significance levels
significance_levels = {}

# Iterate over each column in the DataFrame
for column in selected_returns.columns:
    # Perform the Anderson-Darling normality test on the column
    result = anderson(selected_returns[column])
    
    # Extract the critical values and significance levels
    critical_values = result.critical_values
    significance = result.significance_level
    
    # Find the index at which the null hypothesis can be rejected
    rejection_index = next((i for i, cv in enumerate(critical_values) if result.statistic > cv), None)
    
    # If rejection index is found, find the corresponding significance level
    if rejection_index is not None:
        rejection_level = significance[rejection_index]
    else:
        rejection_level = None
    
    # Store the significance level in the dictionary
    significance_levels[column] = rejection_level
series_modified =  pd.Series(significance_levels).fillna(-1).astype(int) / 100
series_modified[series_modified == -0.01] = np.nan

# Create a DataFrame to store the results
comparison_df = pd.DataFrame({
    ('Return','Mean'): selected_means,
    ('Return','Median'): selected_medians,
    ('Return','Geometric Mean'): geometric_means,
    ('Return','Bayes Stein'): selected_bayes_stein_returns,
    ('Risk','Standard Dev.'): selected_std_dev,
    ('Risk','Skewness'): selected_skewness,
    ('Risk','Kurtosis'): selected_kurtosis,
    ('Risk','CVaR'): selected_cvar,
    ('Normality Tests', 'Jarque-Bera') : selected_jb,
    ('Normality Tests', 'p-value') : selected_jb_p,
    ('Normality Tests', 'Shapiro-Wilk') : selected_sw,
    ('Normality Tests', 'p-value ') : selected_sw_p,
    ('Normality Tests', 'Anderson-Darling') : selected_ad,
    ('Normality Tests', 'sig.') : series_modified
})

comparison_df_save = comparison_df.copy()

# Sort the DataFrame by geometric mean in descending order
comparison_df.sort_values(('Return','Mean'), ascending=False, inplace= True)

# Apply color scale to each column in ascending order
comparison_df = comparison_df.style.background_gradient(axis=0)

comparison_df = comparison_df.set_table_styles([
    {'selector': 'th.col_heading.level0', 'props': [('text-align', 'center')]}
])


# Display the styled DataFrame
display(comparison_df)


#######   ##                                                                                                                                                                                                           
#######    ##                                                                                                                                                                                                          
     ##     ##             #######  #######  #######  #######  #######  #######  #           #     #######           #######     #     #     #  #     #  #        #######  #######     #     #######  #     #  ####### 
     ##     ##             #     #  #     #  #     #     #     #        #     #  #           #     #     #           #           #     ##   ##  #     #  #        #     #     #        #     #     #  ##    #  #       
     ##     ##             #######  #     #  #######     #     ####     #     #  #           #     #     #           #######     #     # # # #  #     #  #        #######     #        #     #     #  # ### #  ####### 
     ##    ##              #        #     #  #    #      #     #        #     #  #           #     #     #                 #     #     #  #  #  #     #  #        #     #     #        #     #     #  #    ##        # 
     ##   ##               #        #######  #    ##     #     #        #######  #######     #     #######           #######     #     #     #  #######  #######  #     #     #        #     #######  #     #  ####### 



# 1) Monte Carlo Simulation - w/o weight restrictions

# Create random portfolio weights and indexes
mean_variance_pairs_un = []
weights_list_un=[]
tickers_list_un=[]

for i in tqdm(range(n_unrestricted)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(selected_returns.columns), n_assets, replace=False)
        #- Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs_un:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs_un.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list_un.append(weights)
        tickers_list_un.append(assets)

        break



# 2) Monte Carlo Simulation - with weight restrictions

# Create random portfolio weights and indexes
mean_variance_pairs_re = []
weights_list_re=[]
tickers_list_re=[]

for i in tqdm(range(n_restricted)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(selected_returns.columns), n_assets, replace=False)

        # Find the index of 'cash' asset
        cash_index = np.where(assets == 'cash')[0]

        #- Choose weights randomly ensuring that they don't go above threshold
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)
        while np.any(weights > max_weight) or (cash_index.size > 0 and weights[cash_index] < cash_min):
            weights = np.random.rand(n_assets)
            weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs_re:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs_re.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list_re.append(weights)
        tickers_list_re.append(assets)

        break
    


# Plot Portfolio Optimization with EF, and allocations

# First set of input parameters
title_name_un = 'Efficient Frontier of Unrestricted Portfolio'
fig1, max_sharpe_html1, min_var_html1, closest_std_html1, retail_html1 = plot_ef(mean_variance_pairs_un, tickers_list_un, weights_list_un, risk_free_rate,retail_return,retail_std,title_name_un)


# Second set of input parameters
title_name_re = 'Efficient Frontier of Restricted Portfolio'
fig2, max_sharpe_html2, min_var_html2, closest_std_html2, retail_html2 = plot_ef(mean_variance_pairs_re, tickers_list_re, weights_list_re, risk_free_rate,retail_return,retail_std,title_name_re)


# Display the subplots
fig1.show()
# Display the combined HTML code
display(HTML(max_sharpe_html1 + min_var_html1 + closest_std_html1 + retail_html1))

fig2.show()
# Display the combined HTML code
display(HTML(max_sharpe_html2 + min_var_html2 + closest_std_html2 + retail_html2))


/var/folders/24/dhffcc6s3hv3z0j3fbnd3tj80000gn/T/ipykernel_8030/4167313461.py:308: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  preqin_data_quarter.index = pd.to_datetime(preqin_data_quarter.index)


The weighting coefficient in Bayes-Stein shrinkage: 0.6491


/Users/johannesschmidt/Documents/Masterarbeit/.conda/lib/python3.10/site-packages/scipy/stats/_morestats.py:1813: UserWarning:

Input data for shapiro has range zero. The results may not be accurate.

/Users/johannesschmidt/Documents/Masterarbeit/.conda/lib/python3.10/site-packages/scipy/stats/_morestats.py:1948: RuntimeWarning:

invalid value encountered in divide



100%|██████████| 10000000/10000000 [2:50:37<00:00, 976.79it/s]  


,Asset_classes,Weights
0,state bonds,33.92%
1,listed foreign public equity,17.69%
2,other equity instruments,16.88%
3,fixed term and savings deposit,12.72%
4,corporate bonds,12.3%
5,cash,2.97%
6,public investment funds,2.73%
7,listed domestic public equity,0.78%
,Asset_classes,Weights
0,fixed term and savings deposit,40.0%


,Asset_classes,Weights
0,state bonds,30.84%
1,corporate bonds,28.9%
2,listed foreign public equity,15.93%
3,cash,12.73%
4,other equity instruments,10.97%
5,listed domestic public equity,0.44%
6,fixed term and savings deposit,0.11%
7,public investment funds,0.09%
,Asset_classes,Weights
0,cash,32.84%


# Save Output as CSV

In [2]:
cache_pairs_preqin_quarter_re = pd.DataFrame(mean_variance_pairs_re)
cache_weights_preqin_quarter_re = pd.DataFrame(weights_list_re)
cache_tickers_preqin_quarter_re = pd.DataFrame(tickers_list_re)

cache_pairs_preqin_quarter_re.to_csv('Var_cash_No_alt_RE_7_pairs_4')
cache_weights_preqin_quarter_re.to_csv('Var_cash_No_alt_RE_7_weights_4')
cache_tickers_preqin_quarter_re.to_csv('Var_cash_No_alt_RE_7_tickers_4')